<a href="https://colab.research.google.com/github/drohe/Twittorials/blob/master/interestingFindsMobileSerpFeatureAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding Interesting Finds Features in Mobile SERPs

First, let's install Selenium and load up the Chrome webdriver!

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
G

##Mount Your Google Drive
In Google Drive, access your Colab Notebooks folder and create a new folder called "Interesting Finds"

Within that folder, you can upload a .csv file of your list of queries to run. Each query should be row. Save the file as "queries.csv" and upload it to your "Colab Notebooks > Interesting Finds" Drive folder.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Once the drive is mounted, select the folder icon in the left menu of the notebook and then open the "gdrive" folder. Expand "My Drive" and then "Colab Notebooks" to ensure that your queries.csv file is saved at the path of '/content/gdrive/My Drive/Colab Notebooks/Interesting Finds/queries.csv'. If not, right click to copy the path and replace it in the "with open" line below.

##Execute the script

In [ ]:
import csv
import os
import requests
import urllib
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### START THE SET UP YOUR CHROME DRIVER
mobile_emulation = {'deviceName': 'Pixel 2'}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
browser = webdriver.Chrome('chromedriver',options=chrome_options)
### END CHROME DRIVER SETUP

### START THE SET UP FOR YOUR OUTPUT
os.mkdir('/content/Interesting Finds/')
outputdir = '/content/Interesting Finds/'
f = open('/content/Interesting Finds/output.csv', "w+", newline="\n", encoding="utf-8-sig")
fw = csv.writer(f)
fw.writerow(["Query","Interesting Finds\?", "Finds Links"])
### END THE OUTPUTS SETUP

##IMPORT YOUR QUERIES.CSV
with open('/content/gdrive/My Drive/Colab Notebooks/Interesting Finds/queries.csv',newline='') as r:
    reader = csv.reader(r)
    queries = list(reader)
print(queries)
### OPEN GOOGLE.COM VIA YOUR CHROME DRIVER (NAMED BROWSER) AND RESIZE THE WINDOW (TIED TO THE PIXEL 2) TO HELP WITH SCREENSHOTS
browser.get('https://www.google.com')
browser.set_window_size(360,640)

### BEGIN THE LOOP THROUGH EACH QUERY OF YOUR QUERIES ARRAY
for query in queries:
  query = str(query[0])
  print('running ' + query)
  e = browser.find_element_by_name("q") #Find the input field to have Selenium type in the query
  e.clear()
  e.send_keys(query) #Selenium
  e = browser.find_element_by_name("q")
  e.send_keys(Keys.RETURN)
  e = browser.find_element_by_name("q")
  e = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.ID, "botstuff"))
    )
  body = browser.find_elements_by_tag_name('body')[0] #This is used for screenshots further down

#Interesting Finds
  if browser.find_elements_by_css_selector('g-card.I7zR5'):
    interestingFinds = True
    findsCode = browser.find_elements_by_css_selector('g-card.I7zR5')
    findsLinks = []
    for card in findsCode:
        links = card.find_elements_by_css_selector('div.mnr-c div div div a')
        for link in links:
            href = link.get_attribute('href')
            amp = False
            if link.get_attribute('class'):
                classes = link.get_attribute('class')
                if 'amp_r' in classes:
                    amp = True
            link = {'link': href, 'amp': amp}
            findsLinks.append(link)
  else:
    interestingFinds = False
    findsLinks = ""

  fw.writerow([query,interestingFinds,findsLinks]) #create row in CSV


### START SCREENSHOTS
  imageName = query.replace(' ','-')
  browser.save_screenshot(outputdir + imageName + '.png')
  totalLength = body.size['height']
  lastClip = totalLength -500
  clipsNeeded = round(lastClip/500)
  i = 0
  clipHeight = 500
  while i < clipsNeeded:
    browser.execute_script('window.scrollTo(360,arguments[0])',clipHeight)
    clipNum = str(i).replace(' ','')
    browser.save_screenshot(outputdir + imageName + clipNum + '.png')
    clipHeight = clipHeight + 500
    i += 1
f.close()
print('ready to zip')
!zip -r /content/interestingFinds.zip '/content/Interesting Finds/'
from google.colab import files
files.download('/content/interestingFinds.zip')


[['\ufeffbest face masks'], ['bourbon'], ['2020 toys'], ['2020 christmas gifts'], ['headsets for zoom']]
running ﻿best face masks
running bourbon
running 2020 toys
running 2020 christmas gifts
running headsets for zoom
ready to zip
  adding: content/Interesting Finds/ (stored 0%)
  adding: content/Interesting Finds/2020-christmas-gifts3.png (deflated 4%)
  adding: content/Interesting Finds/﻿best-face-masks.png (deflated 2%)
  adding: content/Interesting Finds/2020-toys1.png (deflated 3%)
  adding: content/Interesting Finds/headsets-for-zoom2.png (deflated 5%)
  adding: content/Interesting Finds/2020-christmas-gifts.png (deflated 2%)
  adding: content/Interesting Finds/2020-christmas-gifts2.png (deflated 2%)
  adding: content/Interesting Finds/bourbon1.png (deflated 4%)
  adding: content/Interesting Finds/﻿best-face-masks0.png (deflated 4%)
  adding: content/Interesting Finds/2020-christmas-gifts0.png (deflated 2%)
  adding: content/Interesting Finds/2020-christmas-gifts7.png (deflated 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>